In [115]:
import requests
import pandas as pd
import time
import os
from config import token
import sqlite3
import warnings
import calendar
from scipy.spatial import cKDTree

In [132]:
# Path to the .csv.gz file
csv_gz_file_path = "Resources/2013.csv.gz"

# Read the compressed CSV file using Pandas
df = pd.read_csv(csv_gz_file_path, compression='gzip')

# Now the 'df' DataFrame contains the data from the compressed CSV file
df.head()

,AE000041196,20130101,TMAX,250,Unnamed: 4,Unnamed: 5,S,Unnamed: 7
0,AE000041196,20130101,TAVG,176,H,NaN,S,NaN
1,AEM00041194,20130101,TMAX,241,NaN,NaN,S,NaN
2,AEM00041194,20130101,TMIN,157,NaN,NaN,S,NaN
3,AEM00041194,20130101,PRCP,0,NaN,NaN,S,NaN
4,AEM00041194,20130101,TAVG,204,H,NaN,S,NaN


In [133]:
df=df[['AE000041196','20130101','TMAX','250']]

# Rename columns
df = df.rename(columns={
    'AE000041196': 'station',
    '20130101': 'date',
    'TMAX': 'data_type',
    '250': 'value'
})

df.head()

,station,date,data_type,value
0,AE000041196,20130101,TAVG,176
1,AEM00041194,20130101,TMAX,241
2,AEM00041194,20130101,TMIN,157
3,AEM00041194,20130101,PRCP,0
4,AEM00041194,20130101,TAVG,204


In [134]:
# List of values to keep
valid_values = ['TMAX', 'TMIN', 'TAVG', 'PRCP','SNOW','SNWD','AWND']

# Filter the DataFrame
filtered_df = df[df['data_type'].isin(valid_values)]
filtered_df.shape

(30450490, 4)

In [135]:
# Convert 'value' column to float, handling invalid entries as NaN
filtered_df['value'] = pd.to_numeric(filtered_df['value'], errors='coerce')

# Initialize empty lists to store the data
dates = []
stations = []
tmax_values = []
tmin_values = []
tavg_values = []
prcp_values = []
snow_values = []
snwd_values = []
awnd_values = []

# Iterate through the rows of the original DataFrame
for index, row in filtered_df.iterrows():
    date = row['date']
    station = row['station']
    datatype = row['data_type']
    value = row['value']

    if datatype == 'TMAX':
        tmax_values.append(value)
        tmin_values.append(None)
        tavg_values.append(None)
        prcp_values.append(None)
        snow_values.append(None)
        snwd_values.append(None)
        awnd_values.append(None)
    elif datatype == 'TMIN':
        tmax_values.append(None)
        tmin_values.append(value)
        tavg_values.append(None)
        prcp_values.append(None)
        snow_values.append(None)
        snwd_values.append(None)
        awnd_values.append(None)
    elif datatype == 'TAVG':
        tmax_values.append(None)
        tmin_values.append(None)
        tavg_values.append(value)
        prcp_values.append(None)
        snow_values.append(None)
        snwd_values.append(None)
        awnd_values.append(None)
    elif datatype == 'PRCP':
        tmax_values.append(None)
        tmin_values.append(None)
        tavg_values.append(None)
        prcp_values.append(value)
        snow_values.append(None)
        snwd_values.append(None)
        awnd_values.append(None)
    elif datatype == 'SNOW':
        tmax_values.append(None)
        tmin_values.append(None)
        tavg_values.append(None)
        prcp_values.append(None)
        snow_values.append(value)
        snwd_values.append(None)
        awnd_values.append(None)
    elif datatype == 'SNWD':
        tmax_values.append(None)
        tmin_values.append(None)
        tavg_values.append(None)
        prcp_values.append(None)
        snow_values.append(None)
        snwd_values.append(value)
        awnd_values.append(None)
    elif datatype == 'AWND':
        tmax_values.append(None)
        tmin_values.append(None)
        tavg_values.append(None)
        prcp_values.append(None)
        snow_values.append(None)
        snwd_values.append(None)
        awnd_values.append(value)

    dates.append(date)
    stations.append(station)

# Create a new DataFrame
new_data = {
    'date': dates,
    'station': stations,
    'TMAX': tmax_values,
    'TMIN': tmin_values,
    'TAVG': tavg_values,
    'PRCP': prcp_values,
    'SNOW': snow_values,
    'SNWD': snwd_values,
    'AWND': awnd_values
}

new_df = pd.DataFrame(new_data)

# Group by date and station and keep non-null values
grouped_df = new_df.groupby(['date', 'station']).first().reset_index()

grouped_df = grouped_df.dropna(subset=['TMAX', 'TMIN', 'TAVG','PRCP','SNOW','SNWD','AWND']) #'SNOW','SNWD',

grouped_df['date'] = pd.to_datetime(grouped_df['date'], format='%Y%m%d')

# Write the grouped DataFrame to a CSV file
csv_filename = 'Outputs/grouped_df.csv'
grouped_df.to_csv(csv_filename, index=False)

# Print the first few rows of the filtered DataFrame
grouped_df.head()

,date,station,TMAX,TMIN,TAVG,PRCP,SNOW,SNWD,AWND
30689,2013-01-01,USW00012923,206.0,89.0,156.0,1019.0,0.0,0.0,80.0
62793,2013-01-02,USW00012923,89.0,56.0,78.0,61.0,0.0,0.0,74.0
94858,2013-01-03,USW00012923,106.0,50.0,69.0,0.0,0.0,0.0,72.0
126936,2013-01-04,USW00012923,89.0,72.0,84.0,10.0,0.0,0.0,64.0
158553,2013-01-05,USW00012923,83.0,72.0,78.0,130.0,0.0,0.0,64.0


In [136]:
grouped_df.shape

(52946, 9)

In [137]:
stations_df = pd.read_csv('Outputs/full_station_list.csv')
stations_df.head()

,elevation,mindate,maxdate,latitude,name,datacoverage,id,elevationUnit,longitude
0,139.0,1948-01-01,2014-01-01,31.57020,"ABBEVILLE, AL US",0.8813,COOP:010008,METERS,-85.24820
1,239.6,1938-01-01,2015-11-01,34.21096,"ADDISON, AL US",0.5059,COOP:010063,METERS,-87.17838
2,302.1,1940-05-01,1962-03-01,34.41667,"ADDISON CENTRAL TOWER, AL US",0.9658,COOP:010071,METERS,-87.31667
3,172.5,1995-04-01,2015-11-01,33.17835,"ALABASTER SHELBY CO AIRPORT ASOS, AL US",0.8064,COOP:010116,METERS,-86.78178
4,183.8,1949-01-01,1949-12-01,34.68910,"BELLE MINA 2 N, AL US",1.0000,COOP:010117,METERS,-86.88190


In [138]:
# Split 'id' column into two columns
stations_df[['station_code', 'station_name']] = stations_df['id'].str.split(':', expand=True)

# Drop the original 'id' column if needed
stations_df = stations_df.drop(columns=['id'])

In [139]:
stations_df.head()

,elevation,mindate,maxdate,latitude,name,datacoverage,elevationUnit,longitude,station_code,station_name
0,139.0,1948-01-01,2014-01-01,31.57020,"ABBEVILLE, AL US",0.8813,METERS,-85.24820,COOP,010008
1,239.6,1938-01-01,2015-11-01,34.21096,"ADDISON, AL US",0.5059,METERS,-87.17838,COOP,010063
2,302.1,1940-05-01,1962-03-01,34.41667,"ADDISON CENTRAL TOWER, AL US",0.9658,METERS,-87.31667,COOP,010071
3,172.5,1995-04-01,2015-11-01,33.17835,"ALABASTER SHELBY CO AIRPORT ASOS, AL US",0.8064,METERS,-86.78178,COOP,010116
4,183.8,1949-01-01,1949-12-01,34.68910,"BELLE MINA 2 N, AL US",1.0000,METERS,-86.88190,COOP,010117


In [140]:
# Load the stations.csv file into stations_df
# stations_df = pd.read_csv('full_station_list.csv')
grouped_df= pd.read_csv('Outputs/grouped_df.csv')

# Merge the two DataFrames based on 'station' using a left join
grouped_df_detailed = pd.merge(grouped_df, stations_df, 
                               left_on='station', right_on='station_name', how='left')

# Drop the redundant columns (from stations_df)
grouped_df_detailed.drop(columns=['station_name'], inplace=True)

grouped_df_detailed=grouped_df_detailed[['station','name','latitude','longitude','elevation','date','TAVG','TMAX','TMIN','PRCP','SNOW','SNWD','AWND']]

# grouped_df_detailed['date'] = pd.to_datetime(grouped_df_detailed['date'], format='%Y%m%d')

grouped_df_detailed[['station_name', 'state']] = grouped_df_detailed['name'].str.split(',', n=1, expand=True)

# Filter out rows where state doesn't end with 'US'
grouped_df_detailed = grouped_df_detailed[grouped_df_detailed['state'].str.strip().str.endswith('US')]

# Write the grouped DataFrame to a CSV file
csv_filename = 'Outputs/readings.csv'
grouped_df_detailed.to_csv(csv_filename, index=False)

grouped_df_detailed.head()

,station,name,latitude,longitude,elevation,date,TAVG,TMAX,TMIN,PRCP,SNOW,SNWD,AWND,station_name,state
0,USW00012923,"GALVESTON SCHOLES FIELD, TX US",29.27036,-94.86421,1.5,2013-01-01,156.0,206.0,89.0,1019.0,0.0,0.0,80.0,GALVESTON SCHOLES FIELD,TX US
1,USW00012923,"GALVESTON SCHOLES FIELD, TX US",29.27036,-94.86421,1.5,2013-01-02,78.0,89.0,56.0,61.0,0.0,0.0,74.0,GALVESTON SCHOLES FIELD,TX US
2,USW00012923,"GALVESTON SCHOLES FIELD, TX US",29.27036,-94.86421,1.5,2013-01-03,69.0,106.0,50.0,0.0,0.0,0.0,72.0,GALVESTON SCHOLES FIELD,TX US
3,USW00012923,"GALVESTON SCHOLES FIELD, TX US",29.27036,-94.86421,1.5,2013-01-04,84.0,89.0,72.0,10.0,0.0,0.0,64.0,GALVESTON SCHOLES FIELD,TX US
4,USW00012923,"GALVESTON SCHOLES FIELD, TX US",29.27036,-94.86421,1.5,2013-01-05,78.0,83.0,72.0,130.0,0.0,0.0,64.0,GALVESTON SCHOLES FIELD,TX US


In [141]:
grouped_df_detailed.shape

(52750, 15)

In [142]:
wildfire=pd.read_csv('Resources/data.csv')

wildfires=wildfire[['LATITUDE', 'LONGITUDE', 'FIRE_SIZE','NWCG_REPORTING_UNIT_NAME', 'FIRE_SIZE_CLASS', 'FIRE_YEAR',
       'FPA_ID', 'FIRE_CODE', 'NWCG_CAUSE_CLASSIFICATION',
       'NWCG_GENERAL_CAUSE', 'FIRE_NAME', 'DISCOVERY_DATE', 'CONT_DATE',
       'DISCOVERY_TIME', 'CONT_TIME', 'STATE', 'COUNTY', 'FIPS_CODE'
       ]]
wildfires=wildfires[
    (wildfires['FIRE_YEAR'].isin([2013]))
]
wildfires.head()

,LATITUDE,LONGITUDE,FIRE_SIZE,NWCG_REPORTING_UNIT_NAME,FIRE_SIZE_CLASS,FIRE_YEAR,FPA_ID,FIRE_CODE,NWCG_CAUSE_CLASSIFICATION,NWCG_GENERAL_CAUSE,FIRE_NAME,DISCOVERY_DATE,CONT_DATE,DISCOVERY_TIME,CONT_TIME,STATE,COUNTY,FIPS_CODE
1634255,40.391944,-123.061944,0.1,Shasta-Trinity National Forest,A,2013,FS-1512595,EK15,Human,Missing data/not specified/undetermined,POWERLINE,5/10/2013,5/10/2013,1353.0,1450.0,CA,105,6105.0
1634256,38.858056,-120.762222,0.1,Eldorado National Forest,A,2013,FS-1511339,HB15,Human,Missing data/not specified/undetermined,DARLING,1/18/2013,1/18/2013,900.0,1138.0,CA,017,6017.0
1634257,45.786667,-114.386944,0.1,Bitterroot National Forest,A,2013,FS-1514180,EKS4,Natural,Natural,SENTIMENTAL,8/3/2013,8/3/2013,1043.0,1556.0,MT,081,30081.0
1634258,39.152222,-120.233333,0.1,Tahoe National Forest,A,2013,FS-1513526,EK18,Natural,Natural,SHERWOOD,7/3/2013,7/5/2013,1929.0,1457.0,CA,061,6061.0
1634259,37.691667,-108.621944,0.1,San Juan National Forest,A,2013,FS-1518204,EKU7,Natural,Natural,519 C,8/21/2013,8/22/2013,822.0,1100.0,CO,083,8083.0


In [143]:
grouped_df_detailed=pd.read_csv('Outputs/readings.csv')

In [144]:
grouped_df_detailed.columns

Index(['station', 'name', 'latitude', 'longitude', 'elevation', 'date', 'TAVG',
       'TMAX', 'TMIN', 'PRCP', 'SNOW', 'SNWD', 'AWND', 'station_name',
       'state'],
      dtype='object')

In [145]:
#Takes 40-45 minutes to run
warnings.filterwarnings("ignore")

wildfires['DISCOVERY_DATE'] = pd.to_datetime(wildfires['DISCOVERY_DATE'])
wildfires['CONT_DATE'] = pd.to_datetime(wildfires['CONT_DATE'])
grouped_df_detailed['date'] = pd.to_datetime(grouped_df_detailed['date'])

def find_nearest_match(row, df, date_col, lat_col, lon_col):
    date_diff = abs((df[date_col] - row['DISCOVERY_DATE']).dt.days)
    lat_diff = abs(df[lat_col] - row['LATITUDE'])
    lon_diff = abs(df[lon_col] - row['LONGITUDE'])
    total_diff = date_diff + lat_diff + lon_diff
    nearest_idx = total_diff.idxmin()
    return df.loc[nearest_idx]

# Create an empty list to hold the merged rows
merged_rows = []

# Iterate over each row in the filtered wildfires DataFrame
for idx, row in wildfires.iterrows():
    nearest_row = find_nearest_match(row, grouped_df_detailed, 'date', 'latitude', 'longitude')
    merged_row = pd.concat([row, nearest_row])
    merged_rows.append(merged_row)

# Concatenate the list of merged rows into a DataFrame
merged_results = pd.concat(merged_rows, axis=1).T

merged_results=merged_results[['LATITUDE', 'LONGITUDE','COUNTY', 'FIPS_CODE','FIRE_SIZE', 'FIRE_SIZE_CLASS',
       'NWCG_CAUSE_CLASSIFICATION', 'NWCG_GENERAL_CAUSE', 'FIRE_NAME',
       'DISCOVERY_DATE', 'CONT_DATE', 'DISCOVERY_TIME', 'CONT_TIME', 'STATE',
       'station', 'station_name','state', 'latitude', 'longitude', 'elevation', 'date', 'TAVG',
       'TMAX', 'TMIN','PRCP','SNOW','SNWD','AWND']]

csv_filename = 'Outputs/merged_results.csv'
merged_results.to_csv(csv_filename, index=False)

merged_results.head(10)


,LATITUDE,LONGITUDE,COUNTY,FIPS_CODE,FIRE_SIZE,FIRE_SIZE_CLASS,NWCG_CAUSE_CLASSIFICATION,NWCG_GENERAL_CAUSE,FIRE_NAME,DISCOVERY_DATE,...,longitude,elevation,date,TAVG,TMAX,TMIN,PRCP,SNOW,SNWD,AWND
0,40.391944,-123.061944,105,6105.0,0.1,A,Human,Missing data/not specified/undetermined,POWERLINE,2013-05-10,...,-124.1602,6.1,2013-05-10,100.0,133.0,94.0,0.0,0.0,0.0,23.0
1,38.858056,-120.762222,017,6017.0,0.1,A,Human,Missing data/not specified/undetermined,DARLING,2013-01-18,...,-94.86421,1.5,2013-01-18,104.0,139.0,61.0,0.0,0.0,0.0,30.0
2,45.786667,-114.386944,081,30081.0,0.1,A,Natural,Natural,SENTIMENTAL,2013-08-03,...,-114.09376,973.8,2013-08-03,156.0,244.0,78.0,0.0,0.0,0.0,16.0
3,39.152222,-120.233333,061,6061.0,0.1,A,Natural,Natural,SHERWOOD,2013-07-03,...,-119.76829,1342.5,2013-07-03,257.0,378.0,178.0,0.0,0.0,0.0,26.0
4,37.691667,-108.621944,083,8083.0,0.1,A,Natural,Natural,519 C,2013-08-21,...,-108.54081,1470.4,2013-08-21,246.0,339.0,150.0,0.0,0.0,0.0,34.0
5,46.061111,-115.036944,049,16049.0,8.0,B,Natural,Natural,MARTEN,2013-07-21,...,-114.09376,973.8,2013-07-21,238.0,350.0,122.0,0.0,0.0,0.0,21.0
6,41.1475,-122.696389,105,6105.0,0.1,A,Natural,Natural,DAVIS,2013-08-25,...,-122.87702,400.3,2013-08-25,204.0,239.0,161.0,81.0,0.0,0.0,12.0
7,44.214722,-86.325278,101,26101.0,0.2,A,Human,Power generation/transmission/distribution,MAPLE,2013-07-23,...,-86.23685,190.3,2013-07-23,240.0,261.0,133.0,0.0,0.0,0.0,60.0
8,35.325833,-111.950833,005,4005.0,0.1,A,Natural,Natural,CINDER,2013-08-17,...,-115.16343,662.8,2013-08-17,368.0,428.0,300.0,0.0,0.0,0.0,43.0
9,36.6225,-112.085556,005,4005.0,0.1,A,Natural,Natural,ROCK,2013-07-03,...,-115.16343,662.8,2013-07-03,393.0,450.0,333.0,0.0,0.0,0.0,35.0


In [146]:
# Suppress warnings
warnings.filterwarnings("ignore")

# Read the CSV file
merged_results = pd.read_csv('Outputs/merged_results.csv')

# Select specific columns from the DataFrame
us_data = merged_results[['FIRE_NAME', 'STATE', 'FIPS_CODE', 'LATITUDE', 'LONGITUDE', 'DISCOVERY_DATE', 'CONT_DATE', 'station_name','state', 'latitude', 'longitude', 'date',
                                    'NWCG_CAUSE_CLASSIFICATION', 'FIRE_SIZE', 'FIRE_SIZE_CLASS',
                                    'elevation', 'TAVG', 'TMAX', 'TMIN', 'PRCP','SNOW','SNWD','AWND']]

# Rename the columns
us_data.columns = ['FIRE_NAME', 'STATE', 'FIPS_CODE', 'FIRE_LATITUDE', 'FIRE_LONGITUDE', 'FIRE_DATE', 'CONTAIN_DATE', 'CLOSEST_STATION', 'STATION_STATE', 
                             'STATION_LAT', 'STATION_LON', 'READINGS_DATE',
                             'CAUSE_CLASSIFICATION', 'FIRE_SIZE', 'FIRE_SIZE_CLASS',
                             'ELEVATION', 'TAVG', 'TMAX', 'TMIN', 'PRCP','SNOW','SNWD','AWND']

# Convert 'CONTAIN_DATE' and 'FIRE_DATE' columns to datetime
us_data['CONTAIN_DATE'] = pd.to_datetime(us_data['CONTAIN_DATE'])
us_data['FIRE_DATE'] = pd.to_datetime(us_data['FIRE_DATE'])

# Calculate the difference between 'CONTAIN_DATE' and 'FIRE_DATE'
us_data['DAYS_TO_CONTAIN'] = (us_data['CONTAIN_DATE'] - us_data['FIRE_DATE']).dt.days

# Replace any NaN values in 'DAYS_TO_CONTAIN' with 1
us_data['DAYS_TO_CONTAIN'].fillna(0, inplace=True)

# Define float and int columns
float_columns = {
    'FIRE_LATITUDE': float,
    'FIRE_LONGITUDE': float,
    'STATION_LAT': float,
    'STATION_LON': float,
    'FIRE_SIZE': float,
    'ELEVATION': float,
    'TAVG': float,
    'TMAX': float,
    'TMIN': float,
    'PRCP': float
}

int_columns = {
    'DAYS_TO_CONTAIN': int,
}

# Convert columns to the specified data types
us_data = us_data.astype({**float_columns, **int_columns})

# Save the DataFrame to a CSV file
csv_filename = 'Outputs/Annual_data/us_data_2013.csv'
us_data.to_csv(csv_filename, index=False)

# Display the last 20 rows of the DataFrame
us_data.tail(10)


,FIRE_NAME,STATE,FIPS_CODE,FIRE_LATITUDE,FIRE_LONGITUDE,FIRE_DATE,CONTAIN_DATE,CLOSEST_STATION,STATION_STATE,STATION_LAT,...,FIRE_SIZE_CLASS,ELEVATION,TAVG,TMAX,TMIN,PRCP,SNOW,SNWD,AWND,DAYS_TO_CONTAIN
66424,AZ-132984,AZ,4013.0,33.5605,-112.0646,2013-10-15,2013-10-15,TUCSON INTERNATIONAL AIRPORT,AZ US,32.13153,...,A,777.6,204.0,283.0,133.0,0.0,0.0,0.0,26.0,0
66425,AZ-132384,AZ,4013.0,33.6483,-112.0820,2013-02-01,2013-02-01,GALVESTON SCHOLES FIELD,TX US,29.27036,...,A,1.5,157.0,200.0,128.0,0.0,0.0,0.0,27.0,0
66426,AZ-132394,AZ,4013.0,33.5722,-112.1251,2013-02-08,2013-02-08,GALVESTON SCHOLES FIELD,TX US,29.27036,...,A,1.5,176.0,200.0,150.0,0.0,0.0,0.0,52.0,0
66427,AZ-132397,AZ,4013.0,33.4726,-112.1253,2013-02-09,2013-02-09,GALVESTON SCHOLES FIELD,TX US,29.27036,...,A,1.5,174.0,206.0,150.0,0.0,0.0,0.0,61.0,0
66428,AZ-132407,AZ,4013.0,33.4694,-112.0317,2013-02-17,2013-02-17,GALVESTON SCHOLES FIELD,TX US,29.27036,...,A,1.5,152.0,206.0,111.0,0.0,0.0,0.0,59.0,0
66429,AZ-132991,AZ,4013.0,33.6069,-112.1291,2013-10-18,2013-10-18,TUCSON INTERNATIONAL AIRPORT,AZ US,32.13153,...,A,777.6,178.0,278.0,89.0,0.0,0.0,0.0,26.0,0
66430,AZ-132414,AZ,4013.0,33.6099,-111.9906,2013-02-18,2013-02-18,GALVESTON SCHOLES FIELD,TX US,29.27036,...,A,1.5,196.0,228.0,144.0,23.0,0.0,0.0,81.0,0
66431,AZ-132419,AZ,4013.0,33.6404,-112.0475,2013-02-23,2013-02-23,GALVESTON SCHOLES FIELD,TX US,29.27036,...,A,1.5,156.0,189.0,128.0,0.0,0.0,0.0,40.0,0
66432,AZ-132437,AZ,4013.0,33.4096,-111.9823,2013-03-03,2013-03-03,GALVESTON SCHOLES FIELD,TX US,29.27036,...,A,1.5,122.0,172.0,72.0,0.0,0.0,0.0,46.0,0
66433,AZ-133038,AZ,4013.0,33.5488,-112.1497,2013-11-12,2013-11-12,TUCSON INTERNATIONAL AIRPORT,AZ US,32.13153,...,A,777.6,220.0,306.0,122.0,0.0,0.0,0.0,54.0,0
